In [1]:
import dlc_bci as bci

import os

import torch
import numpy as np

from models import *
from callbacks import keep_best_model, store_best_model

from types import SimpleNamespace 

import torch 
from torch import optim
from torch import nn

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [34]:
grid_search_on=[
        ("nb_layers", list(range(1, 7))),
        ("nb_hidden", [np.asscalar(n) for n in np.arange(40, 201, 40)]),
        ("activation", [nn.ReLU, nn.Tanh, nn.ELU]),
        ("weight_decay", [0] + [np.asscalar(wd) for wd in np.logspace(-6, -2, 5, base=np.e)]),
        ("dropout", [np.asscalar(d) for d in np.linspace(0, 0.30, 4)]),
        ("optimizer", [optim.Adam, optim.Adadelta, optim.Adamax]),
        ("nb_layers", list(range(1, 7)))
    ]

[('nb_layers', [1, 2, 3, 4, 5, 6]),
 ('nb_hidden', [40, 80, 120, 160, 200]),
 ('activation',
  [torch.nn.modules.activation.ReLU,
   torch.nn.modules.activation.Tanh,
   torch.nn.modules.activation.ELU]),
 ('weight_decay',
  [0,
   0.0024787521766663594,
   0.006737946999085469,
   0.018315638888734186,
   0.04978706836786395,
   0.1353352832366127]),
 ('dropout', [0.0, 0.09999999999999999, 0.19999999999999998, 0.3]),
 ('optimizer',
  [torch.optim.adam.Adam,
   torch.optim.adadelta.Adadelta,
   torch.optim.adamax.Adamax]),
 ('nb_layers', [1, 2, 3, 4, 5, 6])]

### Load

In [2]:
one_khz=False

train = SimpleNamespace()
train.X, train.y = bci.load(root='./data_bci', one_khz=one_khz)
print(str(type(train.X)), train.X.size())
print(str(type(train.y)), train.y.size())

test = SimpleNamespace()
test.X, test.y = bci.load(root='./data_bci', train=False, one_khz=one_khz)
print(str(type(test.X)), test.X.size())
print(str(type(test.y)), test.y.size())

<class 'torch.Tensor'> torch.Size([316, 28, 50])
<class 'torch.Tensor'> torch.Size([316])
<class 'torch.Tensor'> torch.Size([100, 28, 50])
<class 'torch.Tensor'> torch.Size([100])


- `CNN2D_MaxPool`

In [3]:
X_tr, y_tr = CNN2D_MaxPool.prepare_data(train)
X_tr, y_tr = X_tr.to(device), y_tr.to(device)

X_te, y_te = CNN2D_MaxPool.prepare_data(test)
X_te, y_te = X_te.to(device), y_te.to(device)

In [5]:
# torch.manual_seed(1)
# torch.cuda.manual_seed_all(1)

params = {
    "nb_layers": 3,
    "nb_hidden": 80,
    "activation": nn.ReLU,
    "weight_decay": 0.0024787521766663594,
    "dropout": 0.0,
    "optimizer": optim.Adam,
}
model = CNN2D_MaxPool(**params)
model.to(device)

model.fit(X_tr, y_tr, X_te, y_te, epochs=100, callbacks=[keep_best_model], verbose=False)
print("Test score:", model.score(X_te, y_te))

/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:124: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  sum_loss_train += loss.data[0].item()
/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:131: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  test_loss = self.criterion(self(X_test), y_test).data[0] if compute_test_err else None


0.77

- `CNN_1D_MaxPool`

In [64]:
p1 = model1.parameters()
p2 = model2.parameters()

In [65]:
for p in p1:
    print((p == next(p2)).all())

tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')
tensor(1, dtype=torch.uint8, device='cuda:1')


In [ ]:
torch.manual_seed(1)
model1 = CNN_1D_MaxPool(**params)
model1.to(device)

In [ ]:
torch.manual_seed(1)
model2 = CNN_1D_MaxPool(**params)
model2.to(device)

In [28]:
X_tr, y_tr = CNN_1D_MaxPool.prepare_data(train)
X_tr, y_tr = X_tr.to(device), y_tr.to(device)

X_te, y_te = CNN_1D_MaxPool.prepare_data(test)
X_te, y_te = X_te.to(device), y_te.to(device)

In [35]:
params = {
    "nb_layers": 2,
    "nb_hidden": 40,
    "activation": nn.ReLU,
    "weight_decay": 0, #0.0024787521766663594
    "dropout": 0.01,
    "optimizer": optim.Adam,
}
model = CNN_1D_MaxPool(**params)
model.to(device)

model.fit(X_tr, y_tr, X_te, y_te, epochs=100, callbacks=[keep_best_model], verbose=True)
print("Test score:", model.score(X_te, y_te))

/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:124: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  sum_loss_train += loss.data[0].item()
/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:131: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  test_loss = self.criterion(self(X_test), y_test).data[0] if compute_test_err else None


Epoch 0: Train loss: 11.283170342445374. Train accuracy 49.68%. Test accuracy 51.00%
Epoch 1: Train loss: 11.096117436885834. Train accuracy 57.28%. Test accuracy 54.00%
Epoch 2: Train loss: 11.18070363998413. Train accuracy 49.68%. Test accuracy 51.00%
Epoch 3: Train loss: 11.138029992580414. Train accuracy 51.90%. Test accuracy 51.00%
Epoch 4: Train loss: 11.002616167068481. Train accuracy 57.59%. Test accuracy 61.00%
Epoch 5: Train loss: 11.042902708053589. Train accuracy 58.86%. Test accuracy 55.00%
Epoch 6: Train loss: 10.859645187854767. Train accuracy 63.29%. Test accuracy 59.00%
Epoch 7: Train loss: 10.904100835323334. Train accuracy 64.24%. Test accuracy 58.00%
Epoch 8: Train loss: 10.316790997982025. Train accuracy 70.89%. Test accuracy 57.00%
Epoch 9: Train loss: 8.793445199728012. Train accuracy 66.77%. Test accuracy 60.00%
Epoch 10: Train loss: 11.095050156116486. Train accuracy 63.29%. Test accuracy 59.00%
Epoch 11: Train loss: 8.754759848117828. Train accuracy 71.84%. Te

- `CNN_1D_BatchNorm`

In [9]:
X_tr, y_tr = CNN_1D_BatchNorm.prepare_data(train)
X_tr, y_tr = X_tr.to(device), y_tr.to(device)

X_te, y_te = CNN_1D_BatchNorm.prepare_data(test)
X_te, y_te = X_te.to(device), y_te.to(device)

In [23]:
params = {
    "nb_layers": 3,
    "nb_hidden": 40,
    "activation": nn.Tanh,
    "weight_decay": 0.007, #0.006737946999085469
    "dropout": 0.01,
    "optimizer": optim.Adam,
}
model = CNN_1D_BatchNorm(**params)
model.to(device)

model.fit(X_tr, y_tr, X_te, y_te, epochs=100, callbacks=[keep_best_model], verbose=False)
print("Test score:", model.score(X_te, y_te))

/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:124: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  sum_loss_train += loss.data[0].item()
/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:131: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  test_loss = self.criterion(self(X_test), y_test).data[0] if compute_test_err else None


Test score: 0.78


- `CNN_1D_BatchNorm_Dial`

In [14]:
X_tr, y_tr = CNN_1D_BatchNorm_Dial.prepare_data(train)
X_tr, y_tr = X_tr.to(device), y_tr.to(device)

X_te, y_te = CNN_1D_BatchNorm_Dial.prepare_data(test)
X_te, y_te = X_te.to(device), y_te.to(device)

In [19]:
params = {
    "nb_layers": 1,
    "nb_hidden": 40,
    "activation": nn.Tanh,
    "weight_decay": 0.13, #0.1353352832366127
    "dropout": 0.2,
    "optimizer": optim.Adamax,
}
model = CNN_1D_BatchNorm_Dial(**params)
model.to(device)

model.fit(X_tr, y_tr, X_te, y_te, epochs=100, callbacks=[keep_best_model], verbose=False)
print("Test score:", model.score(X_te, y_te))

/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:124: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  sum_loss_train += loss.data[0].item()
/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:131: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  test_loss = self.criterion(self(X_test), y_test).data[0] if compute_test_err else None


Test score: 0.79


- `CNN_1D_Residual`

In [ ]:
X_tr, y_tr = CNN_1D_Residual.prepare_data(train)
X_tr, y_tr = X_tr.to(device), y_tr.to(device)

X_te, y_te = CNN_1D_Residual.prepare_data(test)
X_te, y_te = X_te.to(device), y_te.to(device)

In [22]:
params = {
    "nb_layers": 2,
    "nb_hidden": 80,
    "activation": nn.Tanh,
    "weight_decay": 0.0025,
    "dropout": 0.01,
    "optimizer": optim.Adadelta,
}
model = CNN_1D_Residual(**params)
model.to(device)

model.fit(X_tr, y_tr, X_te, y_te, epochs=100, callbacks=[keep_best_model], verbose=False)
print("Test score:", model.score(X_te, y_te))

/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:124: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  sum_loss_train += loss.data[0].item()
/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:131: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  test_loss = self.criterion(self(X_test), y_test).data[0] if compute_test_err else None


Test score: 0.75
